This is an example to gather information on a twitter user, store the data in a database and use it to perform analysis. Analyses include classification of messages and network analysis. 

To start, let's define a twitter user whose network we want to explore. For test reasons let's use my twitter account 'fcarrillo81'. 

In [1]:
source_node_name = 'fcarrillo81'

Let's set-up the database storing the information and connect to it. The database is defined in src.db_tables.py and will be filled during the information gathering phase and cotain tables for i) USER, ii) MESSAGE, iii) LOCATION and iv) CONNECTION. All information will be stored in data/'source_node_name'.db and can be accessed using sqlite. 

In [7]:
from src.db_tables import Base, User, Connection, Message, Location, create_sqlite_db
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

# Create the database.
create_sqlite_db(source_node_name)
# Set-up a connection. 
engine = create_engine('sqlite:///data/' + source_node_name + '.db')
Base.metadata.bind = engine 
DBSession = sessionmaker(bind=engine)
session = DBSession()

Now let's set-up the twitter connection. To this end you have to 